In [1]:
from model.lenet import LeNet
from pl_model.dann import DANN

import pytorch_lightning as pl

In [2]:
pl.seed_everything(42)

Global seed set to 42


42

# Model

In [3]:
base_model = LeNet()
params = {
    "batch_size": 64,
    "epoch": 20,
    "lr": 1e-2,
    "src": "mnist",
    "tgt": "mnist-m",
    "img_size": 28,
    "gamma": 10,
    "alpha": 10,
    "beta": 0.75
}
model = DANN(
    model=base_model, 
    params=params
)
logger = pl.loggers.TestTubeLogger("output", name=f"{params['src']}2{params['tgt']}")
logger.log_hyperparams(params)
trainer = pl.Trainer(
    deterministic=True,
    check_val_every_n_epoch=1, 
    gpus=1,
    logger=logger,
    max_epochs=params["epoch"], 
    weights_summary=None,
)
trainer.fit(model)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation sanity check: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validating: |          | 0/? [00:00<?, ?it/s]

Validating: |          | 0/? [00:00<?, ?it/s]

...

Validating: |          | 0/? [00:00<?, ?it/s]

Validating: |          | 0/? [00:00<?, ?it/s]

Validating: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

In [4]:
trainer.test()

Testing: |          | 0/? [00:00<?, ?it/s]

KeyboardInterrupt: 